It is highly recommended to use a powerful **GPU**, you can use it for free uploading this notebook to [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb).
<table align="center">
 <td align="center"><a target="_blank" href="https://colab.research.google.com/github/ezponda/intro_deep_learning/blob/main/class/RNN/img2seq.ipynb">
        <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/img2seq.ipynb">
        <img src="https://i.ibb.co/xfJbPmL/github.png"  height="70px" style="padding-bottom:5px;"  />View Source on GitHub</a></td>
</table>

## Generate CAPTCHA Dataset

In [ ]:
import string
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import tensorflow as tf
from IPython.display import Image, display

In [ ]:
chars = set()
#chars.update(string.ascii_lowercase)
#chars.update(string.ascii_uppercase)
chars.update({str(i) for i in range(10)})
chars = sorted(chars)
print('Number of chars: {0}, chars: {1}'.format(len(chars), chars))

In [ ]:
img_dir = './captcha/'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)

In [ ]:
#!pip install captcha

In [ ]:
from captcha.image import ImageCaptcha
import uuid
create_dataset = True
captcha_len = 3
width = 40 + 20 * captcha_len
height = 100
n_images = 40000
if create_dataset:
    image = ImageCaptcha(width = width, height = height)
    print('Sample captcha str', np.random.choice(chars, captcha_len))
    seen = set()
    for _ in tqdm(range(n_images)):
        combi = np.random.choice(chars, captcha_len)
        captcha = ''.join(combi)
        image.write(captcha, '{0}{1}_{2}.png'.format(img_dir, captcha, uuid.uuid4()))

In [ ]:
print('Number of captchas', len(os.listdir(img_dir)))
print('Some captchas', os.listdir(img_dir)[:5])

In [ ]:
## Plot first sample
name = os.listdir(img_dir)[0]
x = tf.keras.preprocessing.image.load_img(os.path.join(img_dir, name))
x = tf.keras.preprocessing.image.img_to_array(x).astype(np.uint8)
print('image shape: ', x.shape)
plt.imshow(x)
plt.title(name.split('_')[0])
plt.show()

## Preprocessing data

In [ ]:
data_dir = img_dir
# Get list of all the images
images, labels = zip(*[(os.path.join(img_dir, name), name.split('_')[0])
                       for name in os.listdir(img_dir)])
images, labels = (np.array(list(images)), np.array(list(labels)))
characters = sorted(set(char for label in labels for char in label))

print("Number of images found: ", len(images))
print("Number of labels found: ", len(labels))
print("Number of unique characters: ", len(characters))
print("Characters present: ", characters)
display(Image(images[0]))
print('captcha:', labels[0])

### Characters processing
For converting the characters to one-hot encoding, we will use [tf.keras.layers.experimental.preprocessing.StringLookup
](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/StringLookup). 
```python
tf.keras.layers.experimental.preprocessing.StringLookup(
    max_tokens=None, num_oov_indices=1, mask_token='',
    oov_token='[UNK]', vocabulary=None, encoding=None, invert=False,
    **kwargs
)
```

In [ ]:
# Mapping characters to integers
char_to_num = layers.experimental.preprocessing.StringLookup(
    vocabulary=list(characters), num_oov_indices=0, mask_token=None
)

# Mapping integers back to original characters
num_to_char = layers.experimental.preprocessing.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)


train_samples = int(0.7 * len(images))
val_split = int(0.8 * len(images))
x_train, y_train = images[:train_samples], labels[:train_samples]
x_val, y_val = images[train_samples:val_split], labels[train_samples:val_split]
x_test, y_test = images[val_split:], labels[val_split:]
print('x_train, y_train shape: ', x_train.shape, y_train.shape)

def encode_single_sample(img_path, label):
    
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    # 7. Return a dict as our model is expecting two inputs
    zeros = tf.zeros((captcha_len,1))
    return ((img, zeros), label)


### Generate Datasets

In [ ]:
batch_size = 128
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(encode_single_sample)
train_dataset = train_dataset.batch(batch_size).cache().shuffle(50).prefetch(buffer_size=AUTOTUNE)
    
validation_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).map(encode_single_sample)
validation_dataset = validation_dataset.batch(batch_size).cache().prefetch(buffer_size=AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(encode_single_sample)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE).batch(batch_size)

In [ ]:
# plot samples
_, ax = plt.subplots(4, 4, figsize=(10, 10))
for (images_batch, zeros_batch), labels_batch in train_dataset.take(1):
    for i in range(16):
        img = (images_batch[i] * 255).numpy().astype("uint8")
        label = tf.strings.reduce_join(num_to_char(labels_batch[i])).numpy().decode("utf-8")
        ax[i // 4, i % 4].imshow(img, cmap="gray")
        ax[i // 4, i % 4].set_title(label)
        ax[i // 4, i % 4].axis("off")
plt.show()

## Questions 1: Create a model like a seq2seq
Study the impact of `encoder_vec_dim` on the performance of the model. 

In [ ]:
encoder_vec_dim = ...  # dimension of the encoder vector

# Encoder
encoder_input = tf.keras.Input(shape=(width, height, 3), name='encoder_input')

## Convolution + pooling layers
...
## Flatten()
...



# encoded_vector
x = layers.Dense(encoder_vec_dim, activation='relu')(...)
encoded_vector = [x, x]



# Decoder: encoded_vector is the input state to the first decoder RNN
decoder_input = tf.keras.Input(shape=(captcha_len, 1), name='decoder_input')
decoder_output = layers.LSTM(encoder_vec_dim,
                             return_sequences=True,
                             name="decoder")(decoder_input,
                                             initial_state=encoded_vector)
output = layers.TimeDistributed(
    layers.Dense(len(characters) + 1, activation='softmax'))(decoder_output)






model = tf.keras.Model([encoder_input, decoder_input], output)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_shapes=True)

In [ ]:
epochs = 300
early_stopping_patience = 50
# Add early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=early_stopping_patience, restore_best_weights=True
)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
)

### Evaluate Test Results

In [ ]:
results = model.evaluate(test_dataset, verbose=1)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

In [ ]:
_, ax = plt.subplots(4, 4, figsize=(12, 12))
for (images_batch, zeros_batch), labels_batch in test_dataset.take(1):
    y_preds = model.predict([images_batch, zeros_batch]).argmax(-1)
    for i in range(16):
        img = (images_batch[i] * 255).numpy().astype("uint8")
        label = tf.strings.reduce_join(num_to_char(
            labels_batch[i])).numpy().decode("utf-8")
        label_pred = tf.strings.reduce_join(num_to_char(
            y_preds[i])).numpy().decode("utf-8")
        ax[i // 4, i % 4].imshow(img, cmap="gray")
        ax[i // 4,
           i % 4].set_title('real:{0}, pred:{1}'.format(label, label_pred))
        ax[i // 4, i % 4].axis("off")

plt.show()

In [ ]:
#pretrained_model = tf.keras.applications.MobileNetV2(input_shape=(width, height, 3), include_top=False)
#pretrained_model.trainable = False

## Extra: Audio Captcha

In [ ]:
from IPython import display
aud_dir = './captcha_audio/'
if not os.path.exists(aud_dir):
    os.makedirs(aud_dir)

In [ ]:
from captcha.image import ImageCaptcha
from captcha.audio import AudioCaptcha

audio = AudioCaptcha()


In [ ]:
#!pip install wavio

In [ ]:
import uuid
import wavio
captcha_len = 3
n_audios = 50
audio = AudioCaptcha()
print('Sample captcha str', np.random.choice(chars, captcha_len))
seen = set()
for _ in tqdm(range(n_audios)):
    combi = np.random.choice(chars, captcha_len)
    captcha = ''.join(combi)
    captcha_path = '{0}{1}_{2}.wav'.format(aud_dir, captcha, uuid.uuid4())
    audio.write(captcha, captcha_path)
    
    wav = wavio.read(captcha_path).data
    # convert to 16-bits
    max_val = 2 ** 16 - 1 
    wav = (wav / 255.0 * max_val).astype(np.int16)
    wav_tf = tf.audio.encode_wav(
    wav, 8000, name=None
    )
    os. remove(captcha_path)
    tf.io.write_file(
    captcha_path, wav_tf, name=None
    )

In [ ]:
print('Number of captchas', len(os.listdir(aud_dir)))
print('Some captchas', os.listdir(aud_dir)[:5])

In [ ]:
file_path = os.path.join(aud_dir, os.listdir(aud_dir)[0]) 
audio_binary = tf.io.read_file(file_path)
audio, _ = tf.audio.decode_wav(audio_binary)
print(audio.shape)
waveform = tf.squeeze(audio, axis=-1)
print(waveform.shape)
print('captcha',os.listdir(aud_dir)[0].split('_')[0])
display.display(display.Audio(waveform, rate=8000))

In [ ]:
# Get list of all the images
audios, labels = zip(*[(os.path.join(aud_dir, name), name.split('_')[0])
                       for name in os.listdir(aud_dir)])
audios, labels = (np.array(list(audios)), np.array(list(labels)))
characters = sorted(set(char for label in labels for char in label))
print("Number of audios found: ", len(audios))
print("Number of labels found: ", len(labels))
print("Number of unique characters: ", len(characters))
print("Characters present: ", characters)


In [ ]:
# Mapping characters to integers
char_to_num = layers.experimental.preprocessing.StringLookup(
    vocabulary=list(characters), num_oov_indices=0, mask_token=None
)

# Mapping integers back to original characters
num_to_char = layers.experimental.preprocessing.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)



train_samples = int(0.7 * len(audios))
val_split = int(0.8 * len(audios))
x_train, y_train = audios[:train_samples], labels[:train_samples]
x_val, y_val = audios[train_samples:val_split], labels[train_samples:val_split]
x_test, y_test = audios[val_split:], labels[val_split:]
print('x_train, y_train shape: ', x_train.shape, y_train.shape)

def encode_single_sample(aud_path, label):
    
    audio_binary = tf.io.read_file(aud_path)
    audio, _ = tf.audio.decode_wav(audio_binary)
    waveform = tf.squeeze(audio, axis=-1)[:70000]
    
    zero_padding = tf.zeros([70000] - tf.shape(waveform), dtype=tf.float32)

    # Concatenate audio with padding so that all audio clips will be of the
    # same length
    waveform = tf.cast(waveform, tf.float32)
    equal_length = tf.concat([waveform, zero_padding], 0)
    
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    # 7. Return a dict as our model is expecting two inputs
    zeros = tf.zeros((captcha_len,1))
    return ((equal_length, zeros), label)


### Generate Datasets

In [ ]:
batch_size = 128
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(encode_single_sample)
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE).batch(batch_size)
    
validation_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).map(encode_single_sample)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(encode_single_sample)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE).batch(batch_size)

In [ ]:
for (audios_batch, zeros_batch), labels_batch in train_dataset.take(1):
    for i in range(4):
        label = tf.strings.reduce_join(num_to_char(labels_batch[i])).numpy().decode("utf-8")
        print('captcha',label)
        display.display(display.Audio(audios_batch[i], rate=8000))
